In [1]:
# Librerías
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from bs4 import BeautifulSoup as bs
import random
from tqdm import tqdm

# Configurar opciones de Chrome
opts = Options()
opts.add_argument("user-agent=Mozilla/124.0.2 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, como Gecko) Chrome/123.0.6312.105 Safari/537.36")

# Función para configurar el driver de Selenium
def configurar_driver():
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=opts
    )
    return driver

# Función para obtener el HTML de una página
def obtener_html(driver, url):
    driver.get(url)
    sleep(random.uniform(5, 15))  # Espera a que la página cargue completamente
    return driver.page_source

# Función para obtener IDs de alojamientos y sus descripciones
def obtener_ids_y_descripciones(driver):
    soup = bs(driver.page_source, 'lxml')
    ids = [a.get('aria-labelledby').replace('title_', '') for a in soup.find('main', {'class': 'c1ym1fv atm_9s_1txwivl dir dir-ltr'}).find_all('a') if a.get('aria-labelledby') is not None]
    descripciones = [titulo.text for titulo in soup.find_all('div',{'data-testid':'listing-card-title'})]
    return ids, descripciones

# Función para pasar a la siguiente página
def ir_a_siguiente_pagina(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[aria-label="Siguiente"]'))
        )
        next_button.click()
        sleep(random.uniform(5, 15))  # Espera a que la página cargue completamente
        return True
    except Exception as e:
        print(f"No se pudo ir a la siguiente página: {e}")
        return False

# Obtener IDs y descripciones de todas las páginas
def obtener_todos_los_ids_y_descripciones(driver, url):  # Eliminar la limitación de páginas
    driver.get(url)
    sleep(random.uniform(5, 15))
    ids = []
    descripciones = []
    while True:  # Eliminar el límite de páginas para procesar todas las páginas disponibles
        new_ids, new_descripciones = obtener_ids_y_descripciones(driver)
        ids.extend(new_ids)
        descripciones.extend(new_descripciones)
        if not ir_a_siguiente_pagina(driver):
            break
    return ids, descripciones

# Definir las coordenadas geográficas de áreas específicas
coordenadas = {
    'Santo Domingo': [
        {'lat': 18.47294876847267, 'lng': -69.88617773594619}, 
        {'lat': 18.49117190671026, 'lng': -69.96093790975502},
        {'lat': 18.48970301878857, 'lng': -69.82460674323852},
    ],
    'Punta Cana': [
        {'lat': 18.481214292880004, 'lng': -68.40393009520155}, 
        {'lat': 18.683012667860705, 'lng': -68.45290646207143}, 
        {'lat': 18.809369696250148, 'lng': -68.58434304682262}, 
    ],
    'Boca Chica': [
        {'lat': 18.461817048507193, 'lng': -69.60458412860156},
        {'lat': 18.45202989281318,  'lng': -69.60132493883327}, 
        {'lat': 18.453295491903262, 'lng': -69.6257157118777},  
    ],
    'Juan Dolio': [
        {'lat': 18.463996341837376, 'lng': -69.44115577015572},
        {'lat': 18.428709076479972, 'lng': -69.42947227618049},
        {'lat': 18.42989439765175,  'lng': -69.3854934951731},
    ],
    'Puerto Plata': [
        {'lat': 19.7662650549899,   'lng': -70.64675199783098},
        {'lat': 19.780192636428584, 'lng': -70.68625326918152},
        {'lat': 19.819759531576565, 'lng': -70.72575454082458},
    ],
    'La Romana': [
        {'lat': 18.441956950494934, 'lng': -69.03701823853086},
        {'lat': 18.434142939498425, 'lng': -68.967339031178},
        {'lat': 18.42432212488203,  'lng': -68.9267113767664},
    ],
    'Bayahibe': [
        {'lat': 18.369401872552203, 'lng': -68.83834610325884},
        {'lat': 18.353286356708793, 'lng': -68.80807657019339},
        {'lat': 18.249933021581004, 'lng': -68.75130916642604},
    ],
    'Cabarete': [
        {'lat': 19.774587110834926, 'lng': -70.44673945782661},
        {'lat': 19.764624099544942, 'lng': -70.42614827856127},
        {'lat': 19.750734868672083, 'lng': -70.41460184352876},
    ],
    'Sosua': [
        {'lat': 19.752504282307683, 'lng': -70.51857326850418},
        {'lat': 19.762146241310482, 'lng': -70.49870785439114},
        {'lat': 19.779547326144275, 'lng': -70.4668482260576},
    ],
    'Samaná': [
        {'lat': 19.21648875385355,  'lng': -69.33674674534565},
        {'lat': 19.205970739249704, 'lng': -69.3432518776393},
        {'lat': 19.205352011560546, 'lng': -69.32972681844014},
    ]
}

# Función para generar URLs de búsqueda basadas en coordenadas y filtros
def generar_url_con_coordenadas(base_url, lat, lng, filtros=None):
    url = f"{base_url}&sw_lat={lat-0.01}&sw_lng={lng-0.01}&ne_lat={lat+0.01}&ne_lng={lng+0.01}"
    if filtros:
        url += filtros
    return url

# Base URL de búsqueda
base_url_sin_filtro = 'https://es.airbnb.com/s/Rep%C3%BAblica-Dominicana/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-07-01&monthly_length=3&monthly_end_date=2024-10-01&price_filter_input_type=0&channel=EXPLORE&date_picker_type=calendar&query=Rep%C3%BAblica%20Dominicana&place_id=ChIJ9bbx3jiIr44R4PcNBiQPAqY&source=structured_search_input_header&search_type=filter_change&search_mode=regular_search&price_filter_num_nights=5&checkin=2024-10-01&checkout=2024-10-03'

# URLs de búsqueda con filtros
filtros = {
    'Reservacion_Inmediata': '&ib=true',
    'Llegada_Autonoma': '&amenities%5B%5D=51',
    'Cancelacion_Gratuita': '&flexible_cancellation=true',
    'Aceptan_Mascotas': '&pets=1'
}

# Inicializar driver
driver = configurar_driver()

# Función para obtener IDs y descripciones de todas las áreas en una ciudad
def obtener_ids_y_descripciones_por_ciudad(driver, ciudad, coordenadas, filtros):
    ids = {filtro: [] for filtro in filtros}
    descripciones = {filtro: [] for filtro in filtros}
    ids['sin_filtro'] = []
    descripciones['sin_filtro'] = []
    for coord in coordenadas:
        # Obtener IDs y descripciones sin filtro
        url = generar_url_con_coordenadas(base_url_sin_filtro, coord['lat'], coord['lng'])
        new_ids, new_descripciones = obtener_todos_los_ids_y_descripciones(driver, url)
        ids['sin_filtro'].extend(new_ids)
        descripciones['sin_filtro'].extend(new_descripciones)
        # Obtener IDs y descripciones con cada filtro
        for filtro, query in filtros.items():
            url = generar_url_con_coordenadas(base_url_sin_filtro, coord['lat'], coord['lng'], query)
            new_ids, new_descripciones = obtener_todos_los_ids_y_descripciones(driver, url)
            ids[filtro].extend(new_ids)
            descripciones[filtro].extend(new_descripciones)
    return ids, descripciones

# Obtener IDs y descripciones para cada ciudad
ids_por_ciudad = {}
descripciones_por_ciudad = {}
for ciudad, coords in coordenadas.items():
    ids_por_ciudad[ciudad], descripciones_por_ciudad[ciudad] = obtener_ids_y_descripciones_por_ciudad(driver, ciudad, coords, filtros)

# Función para combinar IDs y descripciones y eliminar duplicados
def combinar_ids_y_descripciones(ids_por_ciudad, descripciones_por_ciudad):
    all_ids = set()
    all_descripciones = []
    ids_por_filtro = {filtro: set() for filtro in filtros}  # Para almacenar IDs por filtro
    descripciones_por_filtro = {filtro: [] for filtro in filtros}  # Para almacenar descripciones por filtro

    for ciudad in ids_por_ciudad:
        for filtro in ids_por_ciudad[ciudad]:
            for id, desc in zip(ids_por_ciudad[ciudad][filtro], descripciones_por_ciudad[ciudad][filtro]):
                if id not in all_ids:
                    all_ids.add(id)
                    all_descripciones.append(desc)
                if filtro != 'sin_filtro':
                    ids_por_filtro[filtro].add(id)
                    descripciones_por_filtro[filtro].append(desc)

    return list(all_ids), all_descripciones, ids_por_filtro, descripciones_por_filtro

# Combinar IDs y descripciones de todas las ciudades
ids_combinados, descripciones_combinadas, ids_por_filtro, descripciones_por_filtro = combinar_ids_y_descripciones(ids_por_ciudad, descripciones_por_ciudad)

# Función para extraer detalles del anuncio
def extraer_detalles_anuncio(driver, id, ids_por_filtro, descripciones_combinadas):
    url = f'https://www.airbnb.com/rooms/{id}?check_in=2024-10-01&check_out=2024-10-03&adults=1'
    html = obtener_html(driver, url)
    soup = bs(html, 'lxml')
    
    # Extraer título
    try:
        titulo = soup.find('span', {'class': '_1xxgv6l'}).text
    except Exception as e:
        titulo = None
        print(f"Error al extraer título para {id}: {e}")

    # Extraer ubicación
    try:
        ubicacion_div = soup.find('div', {'class': '_152qbzi'})
        if ubicacion_div:
            ubicacion = ubicacion_div.get_text(strip=True)
        else:
            ubicacion = None
            ubicaciones_h3 = soup.find_all('h3', {'class': 'hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 dir dir-ltr'})
            for elem in ubicaciones_h3:
                texto = elem.get_text(strip=True)
                if "República Dominicana" in texto:
                    ubicacion = texto
                    break
        if not ubicacion:
            ubicacion = 'No encontrado'
    except Exception as e:
        ubicacion = None
        print(f"Error al extraer ubicación para {id}: {e}")

    # Extraer precio
    try:
        precio = soup.find('span', {'class': '_1y74zjx'}).text
    except Exception as e:
        precio = None
        print(f"Error al extraer precio para {id}: {e}")

    # Extraer cantidad de huéspedes, habitaciones, camas, baños
    try:
        items = soup.find_all('li', {'class': 'l7n4lsf atm_9s_1o8liyq_keqd55 dir dir-ltr'})
        data = [item.text.strip().replace('\n', '').replace('\t', '').replace('·', '').replace('  ', ' ') for item in items]

        def extract_value(data, keywords):
            for item in data:
                for keyword in keywords:
                    if keyword in item:
                        parts = item.split()
                        for part in parts:
                            if part.isdigit():
                                return part
            return None

        max_personas = extract_value(data, ['huéspedes'])
        habitaciones = extract_value(data, ['habitación', 'habitaciones'])
        camas = extract_value(data, ['cama', 'camas'])
        baños = extract_value(data, ['baño', 'baños'])

    except Exception as e:
        max_personas = None
        habitaciones = None
        camas = None
        baños = None
        print(f"Error al extraer detalles para {id}: {e}")

    # Extraer información de SuperAnfitrión
    try:
        super_anfitrion = soup.find('div', {'class': 'bkwpcc1 atm_mk_stnw88 atm_6i_12gsa0d atm_n3_myb0kj dir dir-ltr'})
        es_super_anfitrion = "SI" if super_anfitrion else "NO"
    except Exception as e:
        es_super_anfitrion = "NO"
        print(f"Error al extraer super anfitrión para {id}: {e}")

    # Extraer calificaciones de limpieza, veracidad, llegada, comunicación, ubicación, calidad-precio
    try:
        items2 = soup.find_all('div', {'class': 'dir dir-ltr'})
        categories = ['Limpieza', 'Veracidad', 'Llegada', 'Comunicación', 'Ubicación', 'Calidad-precio']
        filtered_items = []

        for item in items2:
            text = item.get_text(strip=True).replace('\n', '').replace('\t', '').replace('·', '').replace('  ', ' ')
            if any(category in text for category in categories):
                filtered_items.append(text)

        # Crear un diccionario para organizar los datos
        data_calificaciones = {category: None for category in categories}

        for text in filtered_items:
            for category in categories:
                if category in text:
                    rating = text.split(category)[-1].strip()
                    data_calificaciones[category] = rating

        limpieza = data_calificaciones['Limpieza']
        veracidad = data_calificaciones['Veracidad']
        llegada = data_calificaciones['Llegada']
        comunicacion = data_calificaciones['Comunicación']
        ubicacion_calif = data_calificaciones['Ubicación']
        calidad_precio = data_calificaciones['Calidad-precio']

    except Exception as e:
        limpieza = None
        veracidad = None
        llegada = None
        comunicacion = None
        ubicacion_calif = None
        calidad_precio = None
        print(f"Error al extraer calificaciones para {id}: {e}")

    try:
        room_type = soup.find('h2',{'class':'hpipapi atm_7l_1kw7nm4 atm_c8_1x4eueo atm_cs_1kw7nm4 atm_g3_1kw7nm4 atm_gi_idpfg4 atm_l8_idpfg4 atm_kd_idpfg4_pfnrn2 dir dir-ltr'}).text
    except Exception as e:
        room_type = None
        print(f"Error al extraer tipo hospedaje para {id}: {e}")

    try:
        # Inicializar variables por si no se encuentran
        calificacion = None
        cantidad_evaluaciones = None

        # Encontrar todos los spans con la clase específica
        review_elements = soup.find_all('span', {'class': 'a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_mk_stnw88 atm_vv_1q9ccgz atm_vy_t94yts dir dir-ltr'})
    
        # Iterar sobre los elementos encontrados
        for review in review_elements:
            text = review.get_text()
        
            # Buscar el texto que coincide con el patrón esperado
            if "Calificado con" in text and "de 5 en" in text:
                parts = text.split()
                calificacion = float(parts[2])  # Extrae la calificación (por ejemplo, 4.97)
                cantidad_evaluaciones = int(parts[-2])  # Extrae la cantidad de evaluaciones (por ejemplo, 30)
                break

    except Exception as e:
        print(f"Ocurrió un error al extraer la información: {e}")

    descripcion_alojamiento = descripciones_combinadas[ids_combinados.index(id)]

    return {
        'ID': id,
        'titulo': titulo,
        'Descripcion_alojamiento': descripcion_alojamiento,
        'tipo_hospedaje': room_type,
        'Ubicacion': ubicacion,
        'precio': precio,
        'max_personas': max_personas,
        'habitaciones': habitaciones,
        'camas': camas,
        'baños': baños,
        'super_anfitrion': es_super_anfitrion,
        'cantidad_evaluaciones': cantidad_evaluaciones,
        'rating_total': calificacion,
        'limpieza': limpieza,
        'veracidad': veracidad,
        'llegada': llegada,
        'comunicacion': comunicacion,
        'ubicacion_calif': ubicacion_calif,
        'calidad_precio': calidad_precio,
        'Reservacion_Inmediata': 1 if id in ids_por_filtro['Reservacion_Inmediata'] else 0,
        'Llegada_Autonoma': 1 if id in ids_por_filtro['Llegada_Autonoma'] else 0,
        'Cancelacion_Gratuita': 1 if id in ids_por_filtro['Cancelacion_Gratuita'] else 0,
        'Aceptan_Mascotas': 1 if id in ids_por_filtro['Aceptan_Mascotas'] else 0
    }

# Crear el DataFrame final
anuncios = []

# Usar tqdm para agregar la barra de progreso
for id in tqdm(ids_combinados, desc="Procesando IDs"):  # Procesar todos los IDs
    try:
        detalles = extraer_detalles_anuncio(driver, id, ids_por_filtro, descripciones_combinadas)
        anuncios.append(detalles)
        sleep(random.uniform(2, 5))  # Agregar un retraso entre las solicitudes para evitar sobrecargar el servidor
    except Exception as e:
        print(f"Error al procesar el anuncio con ID {id}: {e}")

driver.quit()

df_anuncios = pd.DataFrame(anuncios)


No se pudo ir a la siguiente página: Message: 
Stacktrace:
	GetHandleVerifier [0x004BB8E3+45827]
	(No symbol) [0x0044DCC4]
	(No symbol) [0x0034150F]
	(No symbol) [0x003820BC]
	(No symbol) [0x0038216B]
	(No symbol) [0x003BE0F2]
	(No symbol) [0x003A2E44]
	(No symbol) [0x003BC034]
	(No symbol) [0x003A2B96]
	(No symbol) [0x00376998]
	(No symbol) [0x0037751D]
	GetHandleVerifier [0x00774513+2899763]
	GetHandleVerifier [0x007C793D+3240797]
	GetHandleVerifier [0x005413B4+593364]
	GetHandleVerifier [0x005482DC+621820]
	(No symbol) [0x004570A4]
	(No symbol) [0x004537A8]
	(No symbol) [0x00453947]
	(No symbol) [0x004459FE]
	BaseThreadInitThunk [0x75137BA9+25]
	RtlInitializeExceptionChain [0x7717BE7B+107]
	RtlClearBits [0x7717BDFF+191]

No se pudo ir a la siguiente página: Message: 
Stacktrace:
	GetHandleVerifier [0x004BB8E3+45827]
	(No symbol) [0x0044DCC4]
	(No symbol) [0x0034150F]
	(No symbol) [0x003820BC]
	(No symbol) [0x0038216B]
	(No symbol) [0x003BE0F2]
	(No symbol) [0x003A2E44]
	(No symbol) 

Procesando IDs:   0%|          | 24/5458 [05:42<19:02:41, 12.62s/it]

Error al extraer título para 919498213388599588: 'NoneType' object has no attribute 'text'


Procesando IDs:   1%|          | 35/5458 [08:14<20:16:31, 13.46s/it]

Error al extraer precio para 609496273986353508: 'NoneType' object has no attribute 'text'


...

Error al procesar el anuncio con ID 45815590: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.114)
Stacktrace:
	GetHandleVerifier [0x004BB8E3+45827]
	(No symbol) [0x0044DCC4]
	(No symbol) [0x0034150F]
	(No symbol) [0x00339B2B]
	(No symbol) [0x0032EB59]
	(No symbol) [0x0032D953]
	(No symbol) [0x0032D506]
	(No symbol) [0x0032D311]
	(No symbol) [0x0032B739]
	(No symbol) [0x0032BBFB]
	(No symbol) [0x00343CF4]
	(No symbol) [0x003BC717]
	(No symbol) [0x003A2DFC]
	(No symbol) [0x003BC034]
	(No symbol) [0x003A2B96]
	(No symbol) [0x00376998]
	(No symbol) [0x0037751D]
	GetHandleVerifier [0x00774513+2899763]
	GetHandleVerifier [0x007C793D+3240797]
	GetHandleVerifier [0x005413B4+593364]
	GetHandleVerifier [0x005482DC+621820]
	(No symbol) [0x004570A4]
	(No symbol) [0x004537A8]
	(No symbol) [0x00453947]
	(No symbol) [0x004459FE]
	BaseThreadInitThunk [0x75137BA9+25]
	RtlInitializeExceptionChain [0x7717BE7B+107]
	RtlClearBits [0x7717BDFF+191]

Error al procesa

Procesando IDs: 100%|█████████▉| 5455/5458 [19:38:25<00:00, 22.42it/s]

Error al procesar el anuncio con ID 1052650699694041306: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.114)
Stacktrace:
	GetHandleVerifier [0x004BB8E3+45827]
	(No symbol) [0x0044DCC4]
	(No symbol) [0x0034150F]
	(No symbol) [0x00339B2B]
	(No symbol) [0x0032EB59]
	(No symbol) [0x0032D953]
	(No symbol) [0x0032D506]
	(No symbol) [0x0032D311]
	(No symbol) [0x0032B739]
	(No symbol) [0x0032BBFB]
	(No symbol) [0x00343CF4]
	(No symbol) [0x003BC717]
	(No symbol) [0x003A2DFC]
	(No symbol) [0x003BC034]
	(No symbol) [0x003A2B96]
	(No symbol) [0x00376998]
	(No symbol) [0x0037751D]
	GetHandleVerifier [0x00774513+2899763]
	GetHandleVerifier [0x007C793D+3240797]
	GetHandleVerifier [0x005413B4+593364]
	GetHandleVerifier [0x005482DC+621820]
	(No symbol) [0x004570A4]
	(No symbol) [0x004537A8]
	(No symbol) [0x00453947]
	(No symbol) [0x004459FE]
	BaseThreadInitThunk [0x75137BA9+25]
	RtlInitializeExceptionChain [0x7717BE7B+107]
	RtlClearBits [0x7717BDFF+191]

Error

Procesando IDs: 100%|██████████| 5458/5458 [19:38:25<00:00, 12.95s/it]


Error al procesar el anuncio con ID 794034967386004731: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=125.0.6422.114)
Stacktrace:
	GetHandleVerifier [0x004BB8E3+45827]
	(No symbol) [0x0044DCC4]
	(No symbol) [0x0034150F]
	(No symbol) [0x00339B2B]
	(No symbol) [0x0032EB59]
	(No symbol) [0x0032D953]
	(No symbol) [0x0032D506]
	(No symbol) [0x0032D311]
	(No symbol) [0x0032B739]
	(No symbol) [0x0032BBFB]
	(No symbol) [0x00343CF4]
	(No symbol) [0x003BC717]
	(No symbol) [0x003A2DFC]
	(No symbol) [0x003BC034]
	(No symbol) [0x003A2B96]
	(No symbol) [0x00376998]
	(No symbol) [0x0037751D]
	GetHandleVerifier [0x00774513+2899763]
	GetHandleVerifier [0x007C793D+3240797]
	GetHandleVerifier [0x005413B4+593364]
	GetHandleVerifier [0x005482DC+621820]
	(No symbol) [0x004570A4]
	(No symbol) [0x004537A8]
	(No symbol) [0x00453947]
	(No symbol) [0x004459FE]
	BaseThreadInitThunk [0x75137BA9+25]
	RtlInitializeExceptionChain [0x7717BE7B+107]
	RtlClearBits [0x7717BDFF+191]



In [12]:
df_anuncios

,ID,titulo,Descripcion_alojamiento,tipo_hospedaje,Ubicacion,precio,max_personas,habitaciones,camas,baños,...,limpieza,veracidad,llegada,comunicacion,ubicacion_calif,calidad_precio,Reservacion_Inmediata,Llegada_Autonoma,Cancelacion_Gratuita,Aceptan_Mascotas
0,1116127358790269247,Casita Melones,Departamento en Santo Domingo,Alojamiento entero: vivienda rentada en Bayahí...,"Bayahíbe, La Altagracia, República Dominicana",$48 USD,4,2,1,1,...,None,None,None,None,None,None,0,1,1,1
1,992297120667509909,Bed & Free Breakfast in Colonial City SDQ,Habitación en Santo Domingo,"Habitación en Santo Domingo, República Dominicana","Santo Domingo, Distrito Nacional, República Do...",$65 USD,None,None,1,None,...,4.6,4.2,4.6,4.8,4.0,4.0,1,0,0,1
2,46279271,Sugar Hill - Sosua,Departamento en Santo Domingo,"Alojamiento entero: villa en Sosúa, República ...","Sosúa, Puerto Plata, República Dominicana",$600 USD,12,6,6,6,...,None,None,None,None,None,None,0,0,1,1
3,975318985183008135,Barranca Oeste #8 Casa De Campo,Departamento en Santo Domingo,"Alojamiento entero: villa en La Romana, Repúbl...","La Romana, República Dominicana","$2,500 USD",14,7,9,None,...,None,None,None,None,None,None,1,1,0,0
4,714321440139540991,"Apartamento encantador con balcón, cerca de pl...",Departamento en Santo Domingo Este,Alojamiento entero: alojamiento vacacional en ...,"Sosúa, Puerto Plata, República Dominicana",$39 USD,4,2,None,1,...,4.8,4.8,4.9,5.0,4.8,4.8,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5033,1136427873332980486,Acogedor departamento familiar,Residencia en Perla Marina,Alojamiento entero: vivienda rentada en Santo ...,"Santo Domingo Este, Santo Domingo, República D...",$100 USD,5,3,3,2,...,None,None,None,None,None,None,0,0,1,0
5034,52225765,MODERNO Y CÓMODO DEPARTAMENTO,Residencia en Perla Marina,Alojamiento entero: condominio en Santo Doming...,"Santo Domingo Este, Santo Domingo, República D...",$63 USD,6,3,3,2,...,4.7,4.3,4.7,4.8,4.6,4.4,1,1,1,0
5035,1121112172224216967,Estudio - Cofresi Palm Beach & Spa Resort #2,Residencia en Perla Marina,"Habitación en hotel en Puerto Plata, República...","Puerto Plata, Puerto Plata Province, República...",$70 USD,4,1,1,1,...,None,None,None,None,None,None,0,0,1,0
5036,1013646414569117044,Brisa Marina Green Hill.,Residencia en Perla Marina,Habitación privada en bed and breakfast en La ...,"La Aguada, Samaná, República Dominicana",$25 USD,2,1,1,1,...,4.9,4.7,4.7,5.0,4.4,4.7,1,0,1,0


In [11]:
df_anuncios.to_excel('C:/Users/ADMIN/Desktop/Hospedajes_RD.xlsx', index=False)